In [1]:
import copy

import matplotlib.pyplot as plt
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from seqeval.scheme import IOB2
from seqeval.metrics import classification_report,f1_score

In [2]:
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
import warnings
import random

from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import KFold

In [3]:
data_train = pd.read_csv("BC_train_IOB2.txt",sep = '\t')
data_dev = pd.read_csv("BC_dev_IOB2.txt",sep = '\t')

In [4]:
data_train

,Word,Tag,Length,Sentence#
0,@,O,1,0
1,Rhy_QD10,O,8,0
2,yeah,O,4,0
3,irking,O,6,0
4,he,O,2,0
...,...,...,...,...
14779,smell,O,5,704
14780,like,O,4,704
14781,dogs,O,4,704
14782,medicine,O,8,704


In [5]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s["Word"].values.tolist(),
                                                     s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence#").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped[self.n_sent]
            self.n_sent += 1
            return s
        except:
            return None

In [6]:
getter = SentenceGetter(data_train)
dev_getter = SentenceGetter(data_dev)

In [7]:
sentences = [[word[0] for word in sentence] for sentence in getter.sentences]
dev_sentences = [[word[0] for word in sentence] for sentence in dev_getter.sentences]
labels = [[s[1] for s in sent] for sent in getter.sentences]
dev_labels = [[s[1] for s in sent] for sent in dev_getter.sentences]

In [8]:
max_features = 40000
tokenizer = Tokenizer(num_words=max_features, split=' ', oov_token='<unk>')
tokenizer.fit_on_texts(sentences)

In [9]:
word2idx = tokenizer.word_index.copy()
word2idx['<pad>'] = 0
idx2word = {word2idx[i]:i for i in word2idx}

In [10]:
len(word2idx)

1328

In [11]:
tags = list(set(data_train['Tag'].values))
tags.append("<pad>")
tag2idx = {t: i for i, t in enumerate(tags)}
idx2tag = {i: t for i, t in enumerate(tags)}

In [12]:
X_train = tokenizer.texts_to_sequences(sentences)
X_test = tokenizer.texts_to_sequences(dev_sentences)
y_train = [[tag2idx[l] for l in s] for s in labels]
y_test = [[tag2idx[l] for l in s] for s in dev_labels]

In [13]:
for i in X_test[0]:
    print(idx2word[i],end = ' ')

just get on birth control and use two <unk> . oh and do n't have <unk> when <unk> . 

In [14]:
from torch.utils.data import Dataset
from torchcrf import CRF
class NERDataset(Dataset):
    def __init__(self,sentences,labels, word_pad_idx, tag_pad_idx, max_len = 500):
        self.sentences = sentences
        self.labels = labels
        self.word_pad_idx = word_pad_idx
        self.tag_pad_idx = tag_pad_idx
        self.max_len = max_len
    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, index):
        return (self.sentences[index],self.labels[index])
        
    def collate_fn(self, datasets):
        sentences = [dataset[0] for dataset in datasets]
        labels = [dataset[1] for dataset in datasets]
        max_sent = max([len(data) for data in sentences])
        max_len = max([min(len(sentence), self.max_len) for sentence in sentences])
        pad_sentence = []
        pad_label = []
        for sentence,label in zip(sentences,labels):
            
            if len(sentence) > max_len:
#                 print('asd')
                pad_sentence.append(sentence[:max_len])
                pad_label.append(label[:max_len])
            else:
#                 print('zxc')
                pad_sentence.append(sentence+[self.word_pad_idx]*(max_len-len(sentence)))
                pad_label.append(label+[self.tag_pad_idx]*(max_len-len(label)))
        return torch.LongTensor(pad_sentence), torch.LongTensor(pad_label)

In [15]:
class EmbeddedRnn(nn.Module):
    def __init__(self, vocab, hidden_dim, output_vocab, n_layer,word_pad_idx,tag_pad_idx):
        super(EmbeddedRnn, self).__init__()
        self.n_layer = n_layer
        self.embedding_size = 300
        self.hidden_dim = hidden_dim
        self.embedded = nn.Embedding(vocab, self.embedding_size , padding_idx  = word_pad_idx)
        self.lstm = nn.LSTM(self.embedding_size, hidden_dim, num_layers=n_layer,batch_first = True, bidirectional=True)
        self.fc1 = nn.Linear(2 * hidden_dim, output_vocab)
        self.softmax = nn.Softmax(dim=-1)
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()
        self.crf = CRF(num_tags=output_vocab, batch_first = True)
        self.tag_pad_idx = tag_pad_idx
        
    def forward(self, x, hidden,y_tag):
        embedded = self.embedded(x)
        output, hidden = self.lstm(embedded, hidden)
        output = self.fc1(output)
        if y_tag is not None:
            mask = y_tag != self.tag_pad_idx
            crf_out = self.crf.decode(output, mask=mask)
            crf_loss = -self.crf(output, tags=y_tag, mask=mask)
        else:
            crf_out = self.crf.decode(output)
            crf_loss = None
        return crf_out, crf_loss
    
    def initHidden(self, batch_size):
        hidden = Variable(torch.zeros(2 * self.n_layer, batch_size, self.hidden_dim))
        cell = Variable(torch.zeros(2 * self.n_layer, batch_size, self.hidden_dim))
        return [hidden, cell]
#         return hidden
    def init_crf_transitions(tag_names, imp_value=-100):
        crf = CRF(num_tags=n_tags)
        num_tags = len(tag_names)
        for i in range(num_tags):
            tag_name = tag_names[i]
            if tag_name[0] in ("I") or tag_name[0] in ("L") or tag_name == "ENDPAD":
                torch.nn.init.constant_(crf.start_transitions[i], imp_value)
        tag_is = {}
        for tag_position in ("B", "I", "O", 'L','U'):
            tag_is[tag_position] = [i for i, tag in enumerate(tag_names) if tag[0] == tag_position]
        impossible_transitions_position = {
            "O": "I",  
            "O": 'L',
            
            'B': 'U',
            'B': 'I',
            
            'I': 'B',
            'I': 'U',
            'I': 'O',
            
            'L': 'I',
            
            'U': 'I',
            'U': 'L',
        }
        for from_tag, to_tag_list in impossible_transitions_position.items():
            to_tags = list(to_tag_list)
            for from_tag_i in tag_is[from_tag]:
                for to_tag in to_tags:
                    for to_tag_i in tag_is[to_tag]:
                        torch.nn.init.constant_(
                            crf.transitions[from_tag_i, to_tag_i], imp_value
                        )
        # init impossible B and I transitions to different entity types
        impossible_transitions_tags = {
            "B": "I",
            'B': 'L',
            "I": "I",
            'I': 'L',
            'U': 'B',
        }
        for from_tag, to_tag_list in impossible_transitions_tags.items():
            to_tags = list(to_tag_list)
            for from_tag_i in tag_is[from_tag]:
                for to_tag in to_tags:
                    for to_tag_i in tag_is[to_tag]:
                        if tag_names[from_tag_i].split("-")[1] != tag_names[to_tag_i].split("-")[1]:
                            torch.nn.init.constant_(
                                crf.transitions[from_tag_i, to_tag_i], imp_value
                            )

## 使用 trainingset+validset 去做 10-fold CV

In [16]:
from torch.utils.data import TensorDataset, DataLoader ,SubsetRandomSampler ,ConcatDataset
bs = 16
k_folds = 10
kfold = KFold(n_splits=k_folds, shuffle=True)
tr_dataset = NERDataset(X_train,y_train,word2idx['<pad>'],tag2idx['<pad>'])
va_dataset = NERDataset(X_test,y_test,word2idx['<pad>'],tag2idx['<pad>'])
dataset = ConcatDataset([tr_dataset, va_dataset])
word_pad_idx = word2idx['<pad>']
tag_pad_idx = tag2idx['<pad>']
num_epoch = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [17]:
All_Fold_score = []
for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):
    print(f'FOLD {fold}')
    print('--------------------------------')
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    train_dataloader = DataLoader(dataset, batch_size=bs,
                                  collate_fn=tr_dataset.collate_fn,
                                  sampler=train_subsampler)
    valid_dataloader = DataLoader(dataset, batch_size=bs,
                                  collate_fn=va_dataset.collate_fn,
                                  sampler=test_subsampler)  
    model = EmbeddedRnn(len(word2idx), 256, len(tags),2,word_pad_idx,tag_pad_idx)
    optimizer = optim.AdamW(model.parameters(), lr=5e-3)
    model = model.to(device)
    all_loader = {"train" : train_dataloader,
                  "valid" : valid_dataloader}
    Fold_score = []
    for epoch in tqdm(range(num_epoch)):
        all_loss = {
            'train': [],
            'valid': [],
        }
        print('')
        for loader in all_loader:
            predictions , true_labels  = [],[]
            for x, y in all_loader[loader]:
                optimizer.zero_grad()
                x = x.to(device)
                y = y.to(device)
                hidden = model.initHidden(x.size(0))
                hidden[0] = hidden[0].to(device)
                hidden[1] = hidden[1].to(device)
                output, loss = model(x, hidden,y)
                if loader == 'train':
                    loss.backward()
                    optimizer.step()
                all_loss[loader].append(loss.cpu().item()) 
                predictions.extend([[idx2tag[j] for j in i] for i in output])
                for i in y.detach().cpu().numpy():
                    _ = []
                    for j in i:
                        if j != tag_pad_idx:
                            _.append(idx2tag[j])
                    true_labels.append(_)
            print(f'{loader}_loss : {np.mean(np.array(all_loss[loader]))/64}')
            f_ = f1_score(true_labels,predictions,scheme = IOB2)
            print(f'{loader}_F1: {f_}')
            if loader == 'valid':
                Fold_score.append(f_)
    if Fold_score != []:
        All_Fold_score.append(Fold_score)

FOLD 0
--------------------------------


  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

C:\ProgramData\Anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: <pad> seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
 10%|████████▎                                                                          | 1/10 [00:03<00:31,  3.52s/it]

train_loss : 1.0442178495552228
train_F1: 0.453355155482815
valid_loss : 0.2164492905139923
valid_F1: 0.8604651162790699



 20%|████████████████▌                                                                  | 2/10 [00:06<00:24,  3.09s/it]

...

train_loss : 0.0625229413094728
train_F1: 0.9208128941836019
valid_loss : 0.20550512274106345
valid_F1: 0.8780487804878048



 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [00:26<00:02,  2.93s/it]

train_loss : 0.05860645291597947
train_F1: 0.924346629986245
valid_loss : 0.20725572109222412
valid_F1: 0.8789808917197451



100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:29<00:00,  2.91s/it]

train_loss : 0.057276470181734665
train_F1: 0.9316420014094433
valid_loss : 0.21379833420117697
valid_F1: 0.9012345679012346


In [18]:
All_score = len(All_Fold_score[0]) * [0]
for score in All_Fold_score:
    All_score = np.sum([All_score,score], axis = 0)
All_score = np.round((All_score / 10) ,2) 

In [19]:
All_score

array([0.83, 0.88, 0.88, 0.87, 0.87, 0.87, 0.87, 0.87, 0.86, 0.88])

## 單用 trainingset 去做 10-fold CV

In [20]:
All_Fold_score = []
for fold, (train_ids, test_ids) in enumerate(kfold.split(tr_dataset)):
    print(f'FOLD {fold}')
    print('--------------------------------')
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    train_dataloader = DataLoader(dataset, batch_size=bs,
                                  collate_fn=tr_dataset.collate_fn,
                                  sampler=train_subsampler)
    valid_dataloader = DataLoader(dataset, batch_size=bs,
                                  collate_fn=va_dataset.collate_fn,
                                  sampler=test_subsampler)  
    model = EmbeddedRnn(len(word2idx), 256, len(tags),2,word_pad_idx,tag_pad_idx)
    optimizer = optim.AdamW(model.parameters(), lr=5e-3)
    model = model.to(device)
    all_loader = {"train" : train_dataloader,
                  "valid" : valid_dataloader}
    Fold_score = []
    for epoch in tqdm(range(num_epoch)):
        all_loss = {
            'train': [],
            'valid': [],
        }
        print('')
        for loader in all_loader:
            predictions , true_labels  = [],[]
            for x, y in all_loader[loader]:
                optimizer.zero_grad()
                x = x.to(device)
                y = y.to(device)
                hidden = model.initHidden(x.size(0))
                hidden[0] = hidden[0].to(device)
                hidden[1] = hidden[1].to(device)
                output, loss = model(x, hidden,y)
                if loader == 'train':
                    loss.backward()
                    optimizer.step()
                all_loss[loader].append(loss.cpu().item()) 
                predictions.extend([[idx2tag[j] for j in i] for i in output])
                for i in y.detach().cpu().numpy():
                    _ = []
                    for j in i:
                        if j != tag_pad_idx:
                            _.append(idx2tag[j])
                    true_labels.append(_)
            print(f'{loader}_loss : {np.mean(np.array(all_loss[loader]))/64}')
            f_ = f1_score(true_labels,predictions,scheme = IOB2)
            print(f'{loader}_F1: {f_}')
            if loader == 'valid':
                Fold_score.append(f_)
    if Fold_score != []:
        All_Fold_score.append(Fold_score)

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

FOLD 0
--------------------------------



 10%|████████▎                                                                          | 1/10 [00:02<00:22,  2.52s/it]

train_loss : 1.1518160752952098
train_F1: 0.4047856430707876
valid_loss : 0.21394155025482178
valid_F1: 0.875



 20%|████████████████▌                                                                  | 2/10 [00:05<00:20,  2.51s/it]

...

train_loss : 0.05508963316679001
train_F1: 0.9265536723163842
valid_loss : 0.04886829853057861
valid_F1: 0.9343065693430657



 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [00:22<00:02,  2.48s/it]

train_loss : 0.05333574637770653
train_F1: 0.9350441058540497
valid_loss : 0.047375190258026126
valid_F1: 0.9253731343283582



100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:24<00:00,  2.46s/it]

train_loss : 0.05351424887776375
train_F1: 0.9270248596631917
valid_loss : 0.04525509476661682
valid_F1: 0.9185185185185185


In [21]:
All_score = len(All_Fold_score[0]) * [0]
for score in All_Fold_score:
    All_score = np.sum([All_score,score], axis = 0)
All_score = np.round((All_score / 10) ,2) 
All_score

array([0.84, 0.91, 0.92, 0.91, 0.91, 0.9 , 0.9 , 0.91, 0.9 , 0.9 ])

## 全部的 trainingset 訓練 model 後, 透過 validset 評估 model

In [26]:
tr_dataset = NERDataset(X_train,y_train,word2idx['<pad>'],tag2idx['<pad>'])
va_dataset = NERDataset(X_test,y_test,word2idx['<pad>'],tag2idx['<pad>'])
train_dataloader = DataLoader(tr_dataset, batch_size=bs,
                            collate_fn=tr_dataset.collate_fn)
valid_dataloader = DataLoader(va_dataset, batch_size=bs,
                            collate_fn=va_dataset.collate_fn)  
model = EmbeddedRnn(len(word2idx), 256, len(tags),2,word_pad_idx,tag_pad_idx)
optimizer = optim.AdamW(model.parameters(), lr=5e-3)
model = model.to(device)
all_loader = {"train" : train_dataloader,
              "valid" : valid_dataloader}

In [27]:
F_score = []
for epoch in tqdm(range(num_epoch)):
    all_loss = {
    'train': [],
    'valid': [],
    }
    print('')
    for loader in all_loader:
        predictions , true_labels  = [],[]
        for x, y in all_loader[loader]:
            optimizer.zero_grad()
            x = x.to(device)
            y = y.to(device)
            hidden = model.initHidden(x.size(0))
            hidden[0] = hidden[0].to(device)
            hidden[1] = hidden[1].to(device)
            output, loss = model(x, hidden,y)
            if loader == 'train':
                loss.backward()
                optimizer.step()
            all_loss[loader].append(loss.cpu().item()) 
            predictions.extend([[idx2tag[j] for j in i] for i in output])
            for i in y.detach().cpu().numpy():
                _ = []
                for j in i:
                    if j != tag_pad_idx:
                        _.append(idx2tag[j])
                true_labels.append(_)
        print(f'{loader}_loss : {np.mean(np.array(all_loss[loader]))/64}')
        f_ = f1_score(true_labels,predictions,scheme = IOB2)
        print(f'{loader}_F1: {f_}')
        if loader == 'valid':
            F_score.append(f_)

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]


train_loss : 0.9918276329835256
train_F1: 0.44787644787644787


 10%|████████▎                                                                          | 1/10 [00:02<00:24,  2.76s/it]

valid_loss : 0.8534433160509381
valid_F1: 0.603174603174603

train_loss : 0.1488828053077062
train_F1: 0.9070887818306951


 20%|████████████████▌                                                                  | 2/10 [00:05<00:21,  2.74s/it]

...

valid_loss : 0.9949791942323957
valid_F1: 0.6763285024154588

train_loss : 0.051482242345809934
train_F1: 0.9358327325162221


 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [00:25<00:02,  2.81s/it]

valid_loss : 0.9544537237712315
valid_F1: 0.6699507389162561

train_loss : 0.05123677518632677
train_F1: 0.9358974358974359


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:28<00:00,  2.81s/it]

valid_loss : 0.965035447052547
valid_F1: 0.6601941747572816


In [28]:
np.round(np.array(F_score),2)

array([0.6 , 0.7 , 0.68, 0.65, 0.66, 0.67, 0.67, 0.68, 0.67, 0.66])